## Cobertura das unidades de saúde da APS


Este script produz os mapas de cobertua da APS

### Entrada (dados necessários):

``` python
# Gerado pelo arquivo de pré processamento que será disponibilizado em breve

# Gerado pelo script de distâncias
distancias = pd.read_parquet(f'./dados/distancias/{UF}.parquet')
unidades = gpd.read_file('./shapes/unidades_processado.gpkg')

# Dados do IBGE - https://www.ibge.gov.br/geociencias/organizacao-do-territorio/malhas-territoriais/26565-malhas-de-setores-censitarios-divisoes-intramunicipais.html
setores = gpd.read_file('./shapes/setores_ligth_processado.gpkg')
estados = gpd.read_file('./shapes/BR_UF_2022.gpkg')
```

### Saída:

Mapas de um estado ou de todos os estados do Brasil no caminho: `./output/site/mapas/cobertura/`

## Mapas de cobertura por município

In [26]:
import os
import geopandas as gpd
import pandas as pd
pd.set_option('display.max_columns', 200)
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
import locale
locale.setlocale(locale.LC_ALL, '')
from shapely import Point
from tqdm import tqdm
import argparse
from geopy.distance import geodesic
import importlib
import sys
import importlib
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import utils
importlib.reload(utils)
from folium.plugins import MeasureControl, Draw, OverlappingMarkerSpiderfier
from utils import *
path_map = "../output/site/mapas/cobertura"

### Funções auxilires

In [27]:
def distribuir_cadastros(cnes_id, cadastros_disponiveis, distancias_ordenadas, setores_temp, ajuste=False):
    #print("CNES:", cnes_id)
    distancias_atuais = distancias_ordenadas[distancias_ordenadas["CO_CNES"] == cnes_id]
    
    if  distancias_atuais.empty: 
        return
    
    visitados = set()  # Para rastrear setores_temp já completamente preenchidos
    
    setores_alocados = {}
    
    for _, row in distancias_atuais.iterrows():
        if cadastros_disponiveis <= 0:
            break
        setor_atual = setores_temp[setores_temp["id_setor"] == row["ID_SETOR"]]
        
        id_setor_atual = setor_atual['id_setor'].values[0]
                    
        if id_setor_atual in visitados:
            continue

        if not setor_atual.empty:
            populacao_total = setor_atual["populacao"].values[0]
        else:
            print("Nenhum setor encontrado com o id_setor especificado.")
        
        populacao_captada = setor_atual['pop_captada'].values[0]
        populacao_restante = populacao_total - populacao_captada
        
        #print("Setor:", id_setor_atual, " restante:", populacao_restante, "populaçao total:", populacao_total, "captada", populacao_captada," restante cadastros:", cadastros_disponiveis, "distancia", row["distancia"])
    
        
        if cadastros_disponiveis >= populacao_restante:
            # Setor completamente preenchido
            setores_temp.loc[setores_temp['id_setor'] == str(id_setor_atual), 'pop_captada'] += populacao_restante
            cadastros_disponiveis -= populacao_restante
            setores_alocados[id_setor_atual] = populacao_restante
            visitados.add(id_setor_atual) 
        else:
            # Setor parcialmente preenchido
            setores_temp.loc[setores_temp['id_setor'] == str(id_setor_atual), 'pop_captada'] += cadastros_disponiveis

            setores_alocados[id_setor_atual] = cadastros_disponiveis
            cadastros_disponiveis = 0  # Todos os cadastros foram utilizados
            break
            
    return (cnes_id, setores_alocados)

def get_resultado(UF, ajustaCadastros = False):
    
    distancias = get_distancias(UF)
    estabelecimentos = unidades.loc[unidades.id_setor.isin(distancias.ID_SETOR)]
    
    processado = []

    municipios = tqdm(set(estabelecimentos.CO_MUNICIPIO))
    alocacao = []
    sem_cadastro = set()
    for municipio in municipios:
        
        ubs_temp = estabelecimentos.loc[estabelecimentos.CO_MUNICIPIO == municipio]
        setores_temp = setores.loc[setores.co_municipio == municipio]
        for _, row in ubs_temp.iterrows():
            retorno = distribuir_cadastros(row['CO_CNES'], row['PARAMETRO_TOTAL'], distancias, setores_temp)
            if retorno:
                alocacao.append((municipio, row['CADASTROS_TOTAIS'], retorno))
            else:
                # estabelecimento sem cadastros para distribuir
                sem_cadastro.add(row['CO_CNES'])
                pass
                #print(f"Não retornou nada: municipio: {municipio} usb: {row['CO_CNES']} cadastros:{row['PARAMETRO_TOTAL']}")
        processado.append(setores_temp)
    #print('Estabelecimentos sem cadastro:',len(sem_cadastro))
    # Resultado final
    resultado = pd.concat(processado)
    if ajustaCadastros:
        resultado['valor'] = resultado.pop_captada / resultado.populacao_ajustada
    else:
        resultado['valor'] = resultado.pop_captada / resultado.populacao
    #resultado = resultado.drop_duplicates()
    resultado = gpd.GeoDataFrame(resultado, geometry='geometry')
    resultado = resultado.set_crs(setores.crs)
    resultado['geoid'] = resultado.index.astype(str)
    #resultado[['id_setor', 'populacao', 'pop_captada', 'valor']]
    return resultado


### Bases de Dados

In [28]:
shape_municipio = gpd.read_file('../shapes/municipios.gpkg')
shape_municipio.columns = [x.upper() for x in shape_municipio.columns]
shape_municipio = gpd.GeoDataFrame(shape_municipio, geometry='GEOMETRY')
unidades = gpd.read_file('../shapes/unidades_processado.gpkg')
estados = gpd.read_file('../shapes/estados.json')

### Processamento

In [29]:
# Calcula a cobertura municipal
shape_municipio = shape_municipio.merge(unidades.groupby('CO_MUNICIPIO').agg({'PARAMETRO_TOTAL':'sum'}).reset_index(),on='CO_MUNICIPIO', how='left')
shape_municipio['COBERTURA'] = shape_municipio['PARAMETRO_TOTAL']/shape_municipio['POPULACAO']

#Limita a cobertura municipal à 100%
shape_municipio.loc[shape_municipio.COBERTURA>1,'COBERTURA'] = 1

# gera a coluna de strings para o mapa
shape_municipio['FORMATED_POPULACAO'] = [format_population(x) for x in shape_municipio.POPULACAO]
shape_municipio['FORMATED_PARAMETRO_TOTAL'] = [format_population(x) for x in shape_municipio.PARAMETRO_TOTAL]
shape_municipio['FORMATED_COBERTURA'] = [format_percentage(x) for x in shape_municipio.COBERTURA]

### Gerando Mapas

In [30]:
def gerar_mapa_cobertura(UF, export=False):
    resultado = shape_municipio.loc[shape_municipio.SG_UF==UF]
    resultado['geoid'] = resultado.index.astype(str)

    unidades_cnes = get_unidades(UF)
    unidades_cnes = unidades_cnes.groupby(['CO_CNES','NO_FANTASIA','NO_MUNICIPIO','NO_LOGRADOURO','DS_TIPO_UNIDADE']).agg({'CO_EQUIPE':list, 'TP_EQUIPE':list, 'DS_EQUIPE':list,'SG_EQUIPE':list,'CO_AREA':list, 'NO_REFERENCIA':list, 'CADASTROS_ACOMPANHADOS':list,'CADASTROS_VINCULADOS':list, 'CADASTROS_PREVINE':list, 'PARAMETRO_CADASTRAL':list,'ST_EQUIPE_VALIDA':list, 'LATITUDE':'mean', 'LONGITUDE':'mean', 'DENTRO_MUNICIPIO':'max'}).reset_index()
    unidades_cnes['geometry'] = [Point(point[0],point[1]) for point in zip(unidades_cnes.LONGITUDE, unidades_cnes.LATITUDE)]

    estados_uf = estados.loc[estados.SIGLA==UF]
    centroUF = estados_uf.representative_point()#.geometry.centroid.values
    map = folium.Map(location=[centroUF.y,centroUF.x], tiles = 'cartodbpositron', control_scale=True, prefer_canvas=True, zoom_start=12)

    marker_cluster = MarkerCluster(name="Unidades básicas de saúde")
    # cria os estabelecimentos
    for index, cnes in unidades_cnes.iterrows():
        caixa = create_box(cnes, export)
        folium.Marker([cnes.geometry.y, cnes.geometry.x], icon=folium.Icon(color="orange", icon="house-medical", prefix='fa'), popup=caixa, tooltip = cnes.NO_FANTASIA).add_to(marker_cluster)

    # Acrescentando os shapes
    folium.Choropleth(
        geo_data=resultado,
        name='Cobertura',
        data=resultado,
        columns=['geoid', 'COBERTURA'],
        key_on='feature.id',
        fill_color='Blues',
        fill_opacity=0.8,
        line_opacity=0.5,
        line_color='black',
        line_weight=.5,
        smooth_factor=1.0,
        nan_fill_color = "White",
        legend_name= 'Percentual de cobertura'
        ).add_to(map)

    # tooltip dos municipios
    folium.features.GeoJson(resultado, name='Informações dos municípios',
        style_function = lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
        tooltip = folium.features.GeoJsonTooltip(fields=['NO_MACRORREGIONAL','NO_MUNICIPIO','FORMATED_POPULACAO','FORMATED_PARAMETRO_TOTAL','FORMATED_COBERTURA'],
                                                aliases = ['Macrorregião','Município','População residente', 'População coberta', 'Percentual de cobertura'],
                                                labels=True, sticky=True),
        highlight_function = lambda x: {'fillColor': '#ffffff', 'color':'#ffffff', 'fillOpacity': .5, 'weight': 0.1}).add_to(map)

    folium.plugins.Geocoder().add_to(map)

    #map.add_child(MeasureControl())
    folium.LayerControl().add_to(map)

    #Draw(export=True).add_to(map)
    Draw().add_to(map)

    # Add the marker cluster to the map
    marker_cluster.add_to(map)
    folium.LayerControl().add_to(map)
    
    # Display/Export o mapa
    export_map(map, f"{path_map}/{UF}.html", export)
    return map


In [31]:
#Testando um estado
gerar_mapa_cobertura("SE", True)

In [18]:
# Gerando mapas de todos os estados do Brasil
export = True # se estiver True, salva os arquivos na pasta como html
for UF in tqdm(co_uf_ibge.keys()):
    gerar_mapa_cobertura(UF, export=export)

## Mapas de cobertura por macroregiões

In [19]:
import numpy as np
import importlib
import utils
importlib.reload(utils)
from utils import *

In [ ]:
def lista_setores(cnes_id,nivel):
    return distancias.loc[(distancias.CO_CNES == cnes_id)&(distancias.nivel==nivel),'ID_SETOR'].to_list()

def total_setores(cnes_id, nivel):
    return setores_temp.loc[setores_temp.id_setor.isin(lista_setores(cnes_id, nivel)),['populacao','pop_captada','populacao_ajustada']].sum()

def aloca_parcial(setores_nivel, cadastro_disponivel):
    temp = setores_temp.loc[setores_temp['id_setor'].isin(setores_nivel)].copy()
    temp["falta"] = temp["populacao"] - temp["pop_captada"] 
    temp["percent"] = temp["falta"]/temp["falta"].sum()
    temp["recebe"] = np.floor(temp["percent"] * cadastro_disponivel).astype(int) # ser rolar um loop infinito não chega a zero
    temp["pop_captada"] = temp["recebe"]+temp["pop_captada"]

    if (temp["pop_captada"].sum() > temp["populacao"].sum()):
        print("ERRO")
        
    setores_temp.loc[setores_temp['id_setor'].isin(setores_nivel), 'pop_captada'] = temp["pop_captada"]
    
def aloca_todos(setores_nivel):
    setores_temp.loc[setores_temp['id_setor'].isin(setores_nivel), 'pop_captada'] = setores_temp.loc[setores_temp['id_setor'].isin(setores_nivel), 'populacao']    
    
def distribuir_cadastros(cnes_id, cadastros_disponiveis, ajustaCadastros = False ):
    niveis_distancia = distancias.loc[distancias.CO_CNES == cnes_id,'nivel'].unique().tolist()

    if len(niveis_distancia)==0: 
        return
        
    for nivel in niveis_distancia:
        total_do_nivel = total_setores(cnes_id, nivel)
        setores_nivel = lista_setores(cnes_id,nivel)

        if ajustaCadastros:
            populacao_total = int(total_do_nivel["populacao_ajustada"])
        else:
            populacao_total = int(total_do_nivel["populacao"])
            
        populacao_captada = int(total_do_nivel['pop_captada'])
        populacao_restante = populacao_total - populacao_captada

        if cadastros_disponiveis >= populacao_restante:
            #aloca todo mundo e segue para o proximo nível
            aloca_todos(setores_nivel)
            cadastros_disponiveis -= populacao_restante
            
        else:
            #aloca com parcial
            aloca_parcial(setores_nivel, cadastros_disponiveis)
            cadastros_disponiveis = 0
            return

        if cadastros_disponiveis <= 0:
            return

In [21]:
setores = gpd.read_file('../shapes/setores_ligth_processado.gpkg')
setores.columns = [col.lower() for col in setores.columns]
unidades = gpd.read_file('../shapes/unidades_processado.gpkg')
macro = gpd.read_file('../shapes/macro.gpkg')
macro = macro.astype({'CO_MACRORREGIONAL':'Int64'})

lista_macro = macro[['CD_MUN', 'CD_UF', 'CO_MACRORREGIONAL', 'NO_MACRORREGIONAL']].rename(columns={'CD_UF':'SG_UF'})
nome_macro = lista_macro[['CO_MACRORREGIONAL','NO_MACRORREGIONAL']].set_index('CO_MACRORREGIONAL').to_dict()
nome_macro = nome_macro.get('NO_MACRORREGIONAL')

### Exportando os mapas

In [ ]:
coluna_cadastros = 'PARAMETRO_TOTAL'
export=True
ajustaCadastros=False
for UF in tqdm(co_uf_ibge.keys()):#tqdm(["AL"]):#tqdm(co_uf_ibge.keys()):
    for MACRO in tqdm(lista_macro.loc[lista_macro.SG_UF==co_uf_ibge.get(UF),'CO_MACRORREGIONAL'].unique(), leave=False):
        try:
            
            # DADOS DE DISTÂNCIAS
            limites = [2**(n-1) for n in range(1, 7)] 
            limites.insert(0,0) # coloca um zero no início da lista
            distancias = get_distancias(UF)
            distancias['nivel'] = pd.cut(
                distancias['distancia'],
                bins=limites,
                labels=[i for i in range(0, 6)] 
            )
            # DADOS DAS UNIDADES
            unidades = get_unidades(UF, MACRO)
            unidades['ACOMPANHADOS_TOTAIS'] = unidades.groupby('CO_CNES')['CADASTROS_ACOMPANHADOS'].transform('sum')
            unidades['CADASTROS_TOTAIS'] = unidades.groupby('CO_CNES')['CADASTROS_VINCULADOS'].transform('sum')
            unidades['PREVINE_TOTAIS'] = unidades.groupby('CO_CNES')['CADASTROS_PREVINE'].transform('sum')
            unidades['PARAMETRO_TOTAL'] = unidades.groupby('CO_CNES')['PARAMETRO_CADASTRAL'].transform('sum')
            unidades = unidades.groupby(['CO_CNES','CO_MUNICIPIO','LATITUDE','LONGITUDE']).agg({'ACOMPANHADOS_TOTAIS':'mean','CADASTROS_TOTAIS':'mean','PREVINE_TOTAIS':'mean','PARAMETRO_TOTAL':'mean'}).reset_index()
            unidades['geometry'] = [Point(long,lat) for long,lat in zip(unidades['LONGITUDE'], unidades['LATITUDE'])]
            unidades = unidades[['CO_CNES','CO_MUNICIPIO','ACOMPANHADOS_TOTAIS','CADASTROS_TOTAIS','PREVINE_TOTAIS','PARAMETRO_TOTAL','geometry']]
            unidades = gpd.GeoDataFrame(unidades, geometry='geometry')
            unidades = unidades.set_crs(setores.crs)
            unidades['id_setor'] = gpd.sjoin(unidades, setores, how='left', predicate='within')['id_setor']
            unidades = unidades.astype({'ACOMPANHADOS_TOTAIS':'Int64','CADASTROS_TOTAIS':'Int64','PREVINE_TOTAIS':'Int64','PARAMETRO_TOTAL':'Int64'})

            municipios = set(unidades.CO_MUNICIPIO) #tqdm(set(unidades.CO_MUNICIPIO))
            alocacao = []
            processado = []
            for municipio in municipios:   
                ubs_temp = unidades.loc[unidades.CO_MUNICIPIO == municipio]
                setores_temp = setores.loc[setores.co_municipio == municipio]
                for _, row in ubs_temp.iterrows():
                    retorno = distribuir_cadastros(row['CO_CNES'], row[coluna_cadastros], ajustaCadastros)  # Depois testar com os parâmetros        
                    alocacao.append((municipio, row[coluna_cadastros], retorno))
                processado.append(setores_temp)

            # Resultado final
            resultado = pd.concat(processado)
            if ajustaCadastros:
                resultado['valor'] = resultado.pop_captada / resultado.populacao_ajustada
            else:
                resultado['valor'] = resultado.pop_captada / resultado.populacao
            resultado['valor_formatado'] = resultado['valor'].apply(lambda x: f"{x:.2%}")

            resultado = gpd.GeoDataFrame(resultado, geometry='geometry')
            resultado = resultado.set_crs(setores.crs)
            resultado['geoid'] = resultado.index.astype(str)

            unidades_cnes = get_unidades(UF, MACRO)
            unidades_cnes = unidades_cnes.groupby(['CO_CNES','NO_FANTASIA','NO_MUNICIPIO','NO_LOGRADOURO','DS_TIPO_UNIDADE']).agg({'CO_EQUIPE':list, 'TP_EQUIPE':list, 'DS_EQUIPE':list,'SG_EQUIPE':list,'CO_AREA':list, 'NO_REFERENCIA':list,'ST_EQUIPE_VALIDA':list,'CADASTROS_ACOMPANHADOS':list,'CADASTROS_VINCULADOS':list, 'CADASTROS_PREVINE':list, 'PARAMETRO_CADASTRAL':list, 'LATITUDE':'mean', 'LONGITUDE':'mean', 'DENTRO_MUNICIPIO':'max'}).reset_index()
            # Após juntar as médias das coordenadas (que deveriam ser iguais), a geometria é recalculada
            unidades_cnes['geometry'] = [Point(point[0],point[1]) for point in zip(unidades_cnes.LONGITUDE, unidades_cnes.LATITUDE)]
            #carrega o estado para colocar um contorno e definir o centro
            # to do: colocar o representative_point() ao invés do centro da Macro
            centroMacro = macro.loc[macro.CO_MACRORREGIONAL == int(MACRO)].representative_point()#.geometry.centroid.values

            # Criando o mapa com tile de fundo
            map = folium.Map(location=[centroMacro.y,centroMacro.x], tiles = 'cartodbpositron', control_scale=True, prefer_canvas=True, zoom_start=11)
            # Cria p objeto marker cluster 
            marker_cluster = MarkerCluster(name="Unidades básicas de saúde")
            # cria os estabelecimentos
            for index, cnes in unidades_cnes.iterrows():
                caixa = create_box(cnes, export)
                folium.Marker([cnes.geometry.y, cnes.geometry.x], icon=folium.Icon(color="orange", icon="house-medical", prefix='fa'), popup=caixa, tooltip = cnes.NO_FANTASIA).add_to(marker_cluster)
            
            # Acrescentando os shapes
            folium.Choropleth(
                geo_data=resultado,
                name='Cobertura',
                data=resultado,
                columns=['geoid', 'valor'],
                key_on='feature.id',
                fill_color='Blues',
                fill_opacity=0.8,
                line_opacity=0.5,
                line_color='black',
                line_weight=.5,
                smooth_factor=1.0,
                nan_fill_color = "White",
                legend_name= 'Percentual de cobertura'
                ).add_to(map)

            # Criando o layer com os tooltips por cima de tudo
            folium.features.GeoJson(resultado, name='Informações dos setores',
                style_function = lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
                tooltip = folium.features.GeoJsonTooltip(fields=['populacao','pop_captada','valor_formatado'],
                                                        aliases = ['População residente', 'População coberta', 'Percentual de cobertura'],
                                                        labels=True, sticky=False),
                #z_index=450,                                        
                highlight_function = lambda x: {'fillColor': '#ffffff', 'color':'#ffffff', 'fillOpacity': .5, 'weight': 0.1}).add_to(map)
            
            # Add the marker cluster to the map
            marker_cluster.add_to(map)

            folium.LayerControl().add_to(map)

            # Display/Export o mapa
            export_map(map, f"{path_map}/{UF}_{MACRO}.html", export)

            map
        except Exception as e:
            print(f'Problemas em gerar os dados de: {UF} - {MACRO}. \n{e}')